# Part 2: Limits of array-oriented programming

Now that we've seen the advantages of array-oriented programming, let's consider its disadvantages.

<br><br><br>

## Performance

Although computing a mathematical formula on millions of values is a lot faster with NumPy arrays than it is with Python scalars, it's not the fastest way to do it.

In [ ]:
import numpy as np
import numexpr as ne

Quadratic formula (one solution $x$ of $ax^2 + bx + c = 0$), just to have something to calculate:

$$ x = \frac{-b + \sqrt{b^2 - 4ac}}{2a} $$

In [ ]:
def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

In [ ]:
a = np.random.uniform(5, 10, 5_000_000)
b = np.random.uniform(10, 20, 5_000_000)
c = np.random.uniform(-0.1, 0.1, 5_000_000)

<br><br><br>

In [ ]:
%%timeit -n1 -r3

imperative = np.empty_like(c)
for i, (ai, bi, ci) in enumerate(zip(a, b, c)):
    imperative[i] = quadratic_formula(ai, bi, ci)

In [ ]:
%%timeit -n1 -r3

quadratic_formula(a, b, c)

In [ ]:
%%timeit -n1 -r3

ne.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)")

<br><br><br>

Why is NumPy so much slower than NumExpr?

<br><br><br>

Each mathematical operation (`+`, `*`, `np.sqrt`) of the NumPy calculation performs that operation on the whole array before moving on to the next operation. If the arrays are larger than your CPU's L1, L2, and L3 caches, it will be forced to get the data from RAM, and memory transfers are usually slower than computing a whole mathematical formula.

What NumPy is doing is almost like this:

In [ ]:
def pedantic_quadratic_formula(a, b, c):
    tmp1 = np.negative(b)            # -b
    tmp2 = np.square(b)              # b**2
    tmp3 = np.multiply(4, a)         # 4*a
    tmp4 = np.multiply(tmp3, c)      # tmp3*c
    del tmp3
    tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
    del tmp2, tmp4
    tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
    del tmp5
    tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
    del tmp1, tmp6
    tmp8 = np.multiply(2, a)         # 2*a
    return np.divide(tmp7, tmp8)     # tmp7 / tmp8

<br><br><br>

Confirm with a performance test:

In [ ]:
%%timeit -n1 -r3

pedantic_quadratic_formula(a, b, c)

In [ ]:
%%timeit -n1 -r3

quadratic_formula(a, b, c)

_(Caveat: NumPy is able to "fuse" some operations, so it's sometimes a little better than the completely "unfused" `pedantic_quadratic_formula`, but both of the above times are much slower than NumExpr.)_

<br><br><br>

NumExpr, on the other hand, turns the `"(-b + sqrt(b**2 - 4*a*c)) / (2*a)"` expression into something that it can compute quickly (a virtual machine like Python, but a lower-overhead one that only does mathematical operations).

<br><br><br>

We could go even further by actually compiling the expression using Numba:

In [ ]:
import numba as nb

@nb.vectorize
def numba_quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

numba_quadratic_formula(a, b, c)

In [ ]:
%%timeit -n100 -r3

numba_quadratic_formula(a, b, c)

<br><br><br>

Or compile it with JAX (we'll hear more about JAX):

In [ ]:
import jax
jax.config.update("jax_platform_name", "cpu"); jax.config.update("jax_enable_x64", True)

@jax.jit
def jax_quadratic_formula(a, b, c):
    return (-b + jax.numpy.sqrt(b**2 - 4*a*c)) / (2*a)

jax_quadratic_formula(a, b, c)

In [ ]:
%%timeit -n100 -r3

jax_quadratic_formula(a, b, c).block_until_ready()

<br><br><br>

Let's make a scorecard:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

scorecard = [
    ("Pure Python",   10.5e0),
    ("Pedantic NumPy", 149e-3),
    ("NumPy",          144e-3),
    ("NumExpr",       45.4e-3),
    ("Numba",         21.3e-3),
    ("JAX",            103e-3),
]

ax.barh([x for x, y in scorecard[::-1]], [y for x, y in scorecard[::-1]])
ax.grid(axis="x", ls=":", c="gray")
ax.set_axisbelow(True)
ax.set_xscale("log")
ax.set_xlabel("time (seconds), smaller is better")

None

The exact results depend on many things, but there's a general pattern of NumPy being orders of magnitude faster than pure Python, while compiled, "single-pass" implementations are several times faster than NumPy.

<br><br><br>

## Expressiveness

Some algorithms are difficult or impossible to express in array-oriented form. You can usually recognize these by the words "iterate until converged..."

<br><br><br>

### Example 1: a good function

To illustrate, suppose that the SciPy library doesn't exist and you want to compute some special functions, such as the log of the Gamma function,

$$\log\Gamma(z) \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} \Gamma(z) = \int_0^\infty t^{z - 1} e^{-t} \, dt$$

The old way to do this is to dust off a copy of [_Numerical Recipes_](http://numerical.recipes/) and translate the algorithm from Fortran into Python.

![](../img/numerical-recipes.jpg)

In [ ]:
def log_of_gamma(x):
    tmp = x + 5.5
    tmp -= (x + 0.5) * np.log(tmp)
    series = 1.000000000190015
    for i, coefficient in enumerate([
        76.18009172947146, -86.50532032941677, 24.01409824083091,
        -1.231739572450155, 0.1208650973866179e-2, -0.5395239384953e-5,
    ]):
        series += coefficient / (x + i + 1)
    return -tmp + np.log(2.5066282746310005 * series / x)

<br><br><br>

We can compute it by passing scalar values to `log_of_gamma`, and verify its correctness by comparing to SciPy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

In [ ]:
fig, ax = plt.subplots()

xs = np.linspace(0, 10, 10000)[1:]
ys = [log_of_gamma(x) for x in xs]
ax.plot(xs, ys)

None

In [ ]:
abs(ys - scipy.special.loggamma(xs)).max()

<br><br><br>

This special function is easy to "vectorize" (replace scalars with arrays) because every operation is a [closed-form expression](https://en.wikipedia.org/wiki/Closed-form_expression). That includes the `for` loop because it's a loop over a fixed number of terms that could as easily have been written as six lines, one for each `coefficient`.

In [ ]:
log_of_gamma(xs)

<br><br><br>

### Example 2: a bad function

Now consider a special function that can't be vectorized this way: incomplete gamma $P(a, x)$.

$$P(a, x) = \frac{1}{\Gamma(a)} \int_0^x t^{a - 1} e^{-t} \, dt \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} a > 0$$

In [ ]:
SMALL_NUMBER = 3e-7

def incomplete_gamma_P(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta
        if np.absolute(delta) < np.absolute(summation) * SMALL_NUMBER:
            return summation * np.exp(-x + a*np.log(x) - gln)

    raise RuntimeError("did not converge")

<br><br><br>

We can evaluate the function with scalars...

In [ ]:
fig, ax = plt.subplots()

xs = np.linspace(0, 15, 10000)[1:]

ax.plot(xs, [incomplete_gamma_P(0.5, x) for x in xs])
ax.plot(xs, [incomplete_gamma_P(2.0, x) for x in xs])
ax.plot(xs, [incomplete_gamma_P(10, x) for x in xs])

None

In [ ]:
abs(np.asarray([incomplete_gamma_P(0.5, x) for x in xs]) - scipy.special.gammainc(0.5, xs)).max()

In [ ]:
abs(np.asarray([incomplete_gamma_P(2, x) for x in xs]) - scipy.special.gammainc(2, xs)).max()

In [ ]:
abs(np.asarray([incomplete_gamma_P(10, x) for x in xs]) - scipy.special.gammainc(10, xs)).max()

<br><br><br>

But we can't evaluate the function with an array.

In [ ]:
incomplete_gamma_P(2, xs)

<br><br><br>

This error message is because Python can't decide whether to enter the body of the `if` statement, to know if it's time to return a value.

Some array elements may have converged already, while others haven't.
* Should `incomplete_gamma_P` return if any one array element has converged?
* Should `incomplete_gamma_P` keep iterating until all of the array elements have converged?

<br><br><br>

### Two ways to work around the issue

1. Keep iterating over all values in the array, needlessly computing values that have already converged.

In [ ]:
def incomplete_gamma_P_numpy_keep_going(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
abs(incomplete_gamma_P_numpy_keep_going(0.5, xs) - scipy.special.gammainc(0.5, xs)).max()

In [ ]:
abs(incomplete_gamma_P_numpy_keep_going(2, xs) - scipy.special.gammainc(2, xs)).max()

In [ ]:
abs(incomplete_gamma_P_numpy_keep_going(10, xs) - scipy.special.gammainc(10, xs)).max()

<br><br><br>

2. Keep track of which values have already converged and don't needlessly compute those?

In [ ]:
SMALL_NUMBER = 3e-7

def incomplete_gamma_P_numpy_tricky_bookkeeping(a, x):
    gln = log_of_gamma(a)

    delta = np.full(x.shape, 1.0 / a)
    summation = delta.copy()

    not_converged = np.ones(x.shape, np.bool_)

    for i in range(100):
        delta[not_converged] *= x[not_converged] / (a + i + 1)
        summation[not_converged] += delta[not_converged]
        not_converged &= np.absolute(delta) >= np.absolute(summation) * SMALL_NUMBER

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
abs(incomplete_gamma_P_numpy_tricky_bookkeeping(0.5, xs) - scipy.special.gammainc(0.5, xs)).max()

In [ ]:
abs(incomplete_gamma_P_numpy_tricky_bookkeeping(2, xs) - scipy.special.gammainc(2, xs)).max()

In [ ]:
abs(incomplete_gamma_P_numpy_tricky_bookkeeping(10, xs) - scipy.special.gammainc(10, xs)).max()

<br><br><br>

It's certainly _simpler_ to compute too much, but which one is faster?

In [ ]:
%%timeit -n100 -r3

incomplete_gamma_P_numpy_keep_going(2, xs)

In [ ]:
%%timeit -n100 -r3

incomplete_gamma_P_numpy_tricky_bookkeeping(2, xs)

<br><br><br>

The easy one is faster! The tricky bookkeeping required to keep track of which array elements have converged was more expensive than needlessly computing them.

That's not unusual. If you've ever done machine learning with PyTorch, you'd recognize a loop like this:

```python
for epoch in range(1000):
    # tell the optimizer to begin an optimization step
    optimizer.zero_grad()

    # use the model as a prediction function: features → prediction
    predictions = model(features)

    # compute the loss between these predictions and the intended targets
    loss = loss_function(predictions, targets)

    # tell the loss function and optimizer to end an optimization step
    loss.backward()
    optimizer.step()
```

which keeps optimizing all neural network weights `1000` times, regardless of whether the fit has converged. In this line of work, it has become normal to ignore formal measures of convergence and treat the number of steps as a hyperparameter, constant during the fit.

<br><br><br>

On to the [project.ipynb](project.ipynb)!